# Extracting Game Features

*RS Turley 8/12/2020*

It will be helpful to extract information from past games to form predictions. For example, A key feature in any strategy concerns the worth of a ship. In order to make decisions regarding spawning, attack, sacrifice, etc., we need some concept of the value of a ship as measured by its halite production or accomplishments in attacking or defending. This notebook is built to extract features from high level games played by others to estimate the value of a ship given various conditions.

Ths code can also be used to derive helpful data in-game.

### Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import signal
import os
import json
#from kaggle_environments.envs.halite.helpers import *

In [2]:
# to better display 21x21 matrices of floats
np.set_printoptions(linewidth=180,precision=1)

## Main Function

**game_df = create_ship_df_gamelevel(json_filename)**

The function `create_ship_df_gamelevel` produces a Pandas dataframe for the game referenced in the json file in `json_filename`. 

## Helpful Functions

The following functions will be helpful in analyzing game boards and player behavior:

* **Calculations for Distances** - for quick look-ups of cardinal directions and distances between cells

  * **cardinal_distance(start_point, end_point)** calculates the shortest distance between two points with the output being the tuple `(dist_west2east , dist_north2south)`, where the first element represents the number of steps going eastward (negative values mean westward) and the number of steps southward (or negative meaning to the north).

  * **make_cardinal_distance_list()** creates a list with precalculated shortest cardinal distances for all points on the board. 
  
  * **make_total_distance_list(cardinal_distances=cardinal_distance_list, boardsize=21)** creates a list with precalculated total distance for all points.

  * **cardinal_distance_list[start_point][end_point]** is a list, not a function. It is created by `make_cardinal_distance_list` and outputs the cardinal distances as steps east and steps south as explained above. Very fast!
  
  * **total_distance_list[start_point][end_point]** is a list, not a function. It is created by `make_total_distance_list` and outputs the absolute sum of the changes in cardinal distances, i.e. the Manhattan distance. Very fast!
  
* **Calculations for Ship Destinations** - yields the destination cell for movements

  * **destination_cell(start_point, move_distance)** is the reverse of the above functions. The inputs are a start point and a tuple representing steps eastward and southward, `(dist_west2east , dist_north2south)`, and the output is the end point on the map. All points are in the raw index fashion used on the game board.

  * **next_location(start_point,ship_action)** is the destination cell for a ship with requested `ship_action` which can be: `North`, `South`, `East`, `West` (or their first letters `N`,`S`,`E`,`W`). Any other input action, including `None` assumes no move.
  
  * **infer_ship_action(start_point,end_point, cardinal_distance_list=cardinal_distance_list)** does the reverse of `next_location`, revealing the ship action that would take a ship from `start_point` to `end_point`. This function results in an error if the start and end points are more than one distance apart.

* **Objects Within a Distance** - these functions consider the cells or cell contents within a given distance. 

  * **cells_in_distance(location, max_distance)** returns the index references for all cells within a specific distance from the location represented as a cell index

  * **distance_kernel(maxdist)** creates a matrix that can be used in a convulution with the game board that sums all squares with distance `maxdist`

  * **halite_sum_matrix(halitedata, maxdist)** with an input list of halite values for the 441 cells on the board, this function returns the sum of all halite within `maxdist` of each cell. It uses matrix convolution to calculate the sum of objects in a distance for every cell on the board, and it was intended to calculate the quantity of halite within a distance. But can easily be repurposed to calculate the sum of any values that can be represented on the board, like the number of enemy ships within a given distance.

* **Deriving Ship Actions from Board Info** uses the observations data from succesive steps

  * **halite_total_change(new_obs,old_obs)** calculates a dictionary of the total change in halite for each ship_id in both `new_obs` and `old_obs`, where the input variables are the observation dictionaries output by each Halite step
  
  * **last_mining_yield(prev_action,old_obs)** calculates a dictionary of halite quantities mined where `prev_action` is 4-element list of ship action dictionaries for each of the four players and `old_obs` is the observation output dictionary from the previous halite step
  
  * **present_prev_actions(ships_present,new_obs,old_obs)** calculates a 4-element list of ship action dictionaries for each of the four players, using the input list of ship id strings in `ships_present` which must correspond to ships that are present in both `new_obs` and `old_obs`
  
  * **investigate_mia(missing_ship_ids, new_obs, old_obs, verbose)** for the list of ship ids in `missing_ship_ids` that are present in old_obs but not in new_obs, this function investigates the cause of dissapearance and outputs three dictionaries: `missing_ship_last_action` is the best guess for the last action submitted for the missing ship, `missing_ship_killer_team` is the team number (0-3) responsible for the ship's death, and `missing_ship_killer_uid` is the ship id of the ship that destroyed the missing ship. If the ship went missing because of a CONVERT action to create a new shipyard or becaues of a suicide mission to destroy an enemy shipyard, the killer team and killer ship id are assigned to be the ship's own team and id.
  
  * **infer_previous_action(new_obs, old_obs, verbose)** this is the main function to infer ship actions for all ships in old_obs using the above functions to assign actions to ships that are still present as well as those that are now missing. The output is a four-element list of dictionaries, one for each team, containing the ship id as as the key and the ship action as the value. 

* **A Code to Describe Board Patterns withing Distance of Three**

  * **nearby_features(location, max_distance)** returns a 25-element `nearby_features_array` with the features labeled in the description above. For example, a value of 2 in array index 3 indicates that a friendly ship in immediately south
  * **rotational_equivalents(nearby_features_array)** returns the 8 features arrays that are rotationally equivalent to the `nearby_features_array`
  * **nearby_feature_code(nearby_features_array)** transforms a 25-element `nearby_features_array` to a unique 64-bit integer




### Functions to create pattern recognition scheme for distance of 3 around a ship

The space around a ship has 24 cells within a distance of three. We will number them in the order below so that the smallest numbers represent the closest distance. The chart below shows the array indices used in saving the information.

 

|   |   |   |   |   |   |   |
|---|---|---|---|---|---|---|
|   |   |   |13 |   |   |   |
|   |   |24 | 5 |14 |   |   |
|   |23 |12 | 1 | 6 |15 |   |
|22 |11 | 4 | 0 | 2 | 7 |16 |
|   |21 |10 | 3 | 8 |17 |   |
|   |   |20 | 9 |18 |   |   |
|   |   |   |19 |   |   |   |


The features in this pattern will define the ships or shipyards that could be present:

0. None
1. Friendly shipyard
2. Friendly ship
3. Enemy shipyard
4. "Fat" enemy ship carrying more halite
5. Lean enemy ship carrying less or equal halite

Since location zero has the ship in question, we will focus on the contents of the other 24 spaces within a distance of three. Every pattern can be represented by an integer between zero and 6^24 (approximately equal to the maximum 64 bit "big int" 2^63). We note that these patterns are functionally identical by symmetry and reflection. The following functions make these calculations:

* **nearby_features(location, max_distance)** returns a 25-element `nearby_features_array` with the features labeled in the description above. For example, a value of 2 in array index 3 indicates that a friendly ship in immediately south
* **rotational_equivalents(nearby_features_array)** returns the 8 features arrays that are rotationally equivalent to the `nearby_features_array`
* **nearby_feature_code(nearby_features_array)** transforms a 25-element `nearby_features_array` to a unique 64-bit integer

## Classes and Functions for Handling Episode Data

The following classes help with the processing of episode data as we create dataframe.
* **BoardOutput(new_obs)** contains properties of the current state of the board as initialized by the game's observation dictionary
* **TeamOutput(new_obs)** contains propeties of the characteristics of the four teams on the board; initialzed with observation
* **ShipOutput(new_obs, include_future)** contains properties describing each of the ships in the game, initialized with observation
* **CumulativeData(new_obs)** contains properties that accumulate over the course of the game. With each successive step, you need to use the method `CumulativeData.update(new_obs)` to accumulate the data from the new observation
* **create_step_features(new_obs, cumulative_data, include_future = False)** function that takes the current game observations object and the cumulative data and outputs a dictionary with all of the features that we intend to calculate for each ship in the step. The option `include_future` allows for the creation of future variables that would not have been known at the time but that we want to capture in the learning process
* **game_df_update(game_df, step_df, cumulative_data)** this function does the update step that takes a pandas dataframe created from the dictionary in `create_step_features()`, and then it appends that to the larger dataframe with all the accumulated data from previous steps that is being stored in `game_df`. If the options `include_future` is True, it will also include and update the future variables as part of the process.

## Build data for an episode

The target variable we hope to predict is ship value. This is defined as the quantity of halite delivered to shipyards, with an additional credit of +500 for destroying a shipyard or saving a friendly shipyard

We could derive a formula or calibrate this to actual game data as a function of the following factors:

Factors that affect ship value include the steps left in the game, halite on board, etc. Our complete list of features is below:


| Type of Variable           | Name                 | type   | description                                                      |
| -------------------------- | -------------------- | ------ | ---------------------------------------------------------------- |
| Identification             | episode              | int    | episode ID number                                                |
|                            | step                 | it     | step number (0-399)                                              |
|                            | agentID              | int    | agentID number for this step                                     |
|                            | shipID               | string | shipID string used in game                                       |
|                            | shipNumID            | int    | alternate numerical ID for each ship                             |
|                            | shipLastAction       | string | N(orth),S(south),E(ast),W(est),C(onvert),M(ine)                  |
|                            | shipNextAction       | string | N(orth),S(south),E(ast),W(est),C(onvert),M(ine)                  |
| Achievement                | futureLife           | int    | number of future steps ship stays alive                          |
|                            | futureHalite         | float  | quantity of halite delivered to friendly base                    |
|                            | futureShipKills      | int    | number of enemy ships destroyed                                  |
|                            | futureShipyardKill   | int    | \=1 if destroyed enemy shipyard                                  |
|                            | futureConvertOne     | int    | \=1 if created shipyard when none present                        |
|                            | futureConvertMore    | int    | \=1 if created shipyard when already present                     |
|                            | futureProtect        | int    | number of times killed enemy attacking friendly shipyard         |
| Game Status                | stepsRemaining       | int    | steps left until game over                                       |
|                            | boardHaliteTotal     | float  | sum total of all halite on game board                            |
|                            | boardShipTotal       | int    | sum total of all ships on game board                             |
| Team Status                | teamHaliteTotal      | int    | current team halite total                                        |
|                            | teamCargo            | int    | current halite total on team ships                               |
|                            | teamShipyards        | int    | number of team shipyards                                         |
|                            | teamShips            | int    | number of team ships                                             |
|                            | teamShipKills        | int    | number of enemy ships killed by team                             |
|                            | teamShipyardKills    | int    | number of enemy shipyards killed by team                         |
| Ship Status                | shipLastAction       | char   | N(orth),S(south,E(ast),W(est),C(onvert),M(ine)                   |
|                            | shipCargo            | float  | amount of halite currently in ship cargo                         |
|                            | shipDistShipyard     | int    | distance to nearest friendly shipyard                            |
|                            | shipEnemiesBlocking  | int    | number of lean enemies in path to friendly shipyard              |
|                            | shipDistEnemyYard    | int    | distance to nearest enemy shipyard                               |
| -- Halite Near             | shipHaliteDist0      | float  | total quantity of halite at distance 0                           |
|                            | shipHaliteDist1      | float  | total quantity of halite at distance 1                           |
|                            | shipHaliteDist2      | float  | total quantity of halite at distance 2                           |
|                            | shipHaliteDist3      | float  | total quantity of halite at distance 3                           |
|                            | shipHaliteDist4      | float  | total quantity of halite at distance 4                           |
|                            | shipHaliteDist5      | float  | total quantity of halite at distance 5                           |
|                            | shipHaliteDist6      | float  | total quantity of halite at distance 6                           |
| -- Friendlies Near         | shipFriendlyDist1    | int    | sum of friendly ships at distance 1                              |
|                            | shipFriendlyDist2    | int    | sum of friendly ships at distance 2                              |
|                            | shipFriendlyDist3    | int    | sum of friendly ships at distance 3                              |
|                            | shipFriendlyDist4    | int    | sum of friendly ships at distance 4                              |
| -- Fat Enemies Near        | shipFatEnemyDist1    | int    | number enemy ships carrying more halite at distance 1            |
|                            | shipFatEnemyDist2    | int    | number enemy ships carrying more halite at distance 2            |
|                            | shipFatEnemyDist3    | int    | number enemy ships carrying more halite at distance 3            |
|                            | shipFatEnemyDist4    | int    | number enemy ships carrying more halite at distance 4            |
| -- Lean Enemies Near       | shipLeanEnemyDist1   | int    | number enemy ships carrying less halite at distance 1            |
|                            | shipLeanEnemyDist2   | int    | number enemy ships carrying less halite at distance 2            |
|                            | shipLeanEnemyDist3   | int    | number enemy ships carrying less halite at distance 3            |
|                            | shipLeanEnemyDist4   | int    | number enemy ships carrying less halite at distance 4            |
|                            | shipLeanBlocking     | int    | number enemy ships carrying less halite blocking path to shipyard|
| -- Shipyard Seige          | shipWeakSeige        | int    | \=1 if within dist=1 enemy yard, enemy halite<500, no lean enemy |
|                            | shipStrongSeige      | int    | \=1 if within dist=1 enemy yard, enemy halite>500 or lean enemy  |
|                            | shipDefendSeige      | int    | \=1 if within dist=1 friendly yard, enemy dist=1 to friendly yard|
| -- Board Pattern           | shipPattern          | int    | 64-bit integer representing the local pattern of the board       |



In [3]:
# make a distance list that preserves the shortest travel route


def cardinal_distance(start_point,end_point,boardsize=21):
    # returns the distance needed to travel across a wrapped board of size [boardsize] where the 
    # first output is the west to east distance (or a negative value if faster to travel westbound)
    # and the second output is the north to south distance (or a negative value if shorter to 
    # travel southbound.
    #
    # The inputs, start_point and end_point are expected to be integers where value zero is the northwest
    # point on the board and value boardsize*boardsize-1 is the southeast point on the board.
    
    # Calculate the distance traveling east (1st element) or west (2nd element)
    dist_west2east = ((end_point - start_point) % boardsize, 
                      (boardsize - ( (end_point - start_point) % boardsize) ))
    # return the signed minimum distance, negative values means travel west
    dist_west2east = min(dist_west2east)*(-1)**dist_west2east.index(min(dist_west2east))

    # Calculate the distance traveling south (1st element) or north (2nd element)
    dist_north2south = ((end_point//boardsize - start_point//boardsize) % boardsize, 
                        ( boardsize - ( (end_point//boardsize - start_point//boardsize) % boardsize) ))
    # return the signed minimum distance, where negative values mean traveling north
    dist_north2south = min(dist_north2south)*(-1)**dist_north2south.index(min(dist_north2south))

    return dist_west2east, dist_north2south

def make_cardinal_distance_list(boardsize=21):
    startpoints = np.arange(boardsize**2)
    endpoints = np.arange(boardsize**2)
    cardinal_distance_list = []
    for start_point in startpoints:
        cardinal_distance_list.append([cardinal_distance(start_point,end_point) for end_point in endpoints])
    return cardinal_distance_list
cardinal_distance_list = make_cardinal_distance_list()

def make_total_distance_list(cardinal_distances=cardinal_distance_list, boardsize=21):
    startpoints = np.arange(boardsize**2)
    endpoints = np.arange(boardsize**2)
    total_distance_list = []
    for start_point in startpoints:
        total_distance_list.append([(abs(cardinal_distances[start_point][end_point][0]) + 
                                     abs(cardinal_distances[start_point][end_point][1]))
                                    for end_point in endpoints])
    return total_distance_list
total_distance_list = make_total_distance_list(cardinal_distance_list)

def destination_cell(start_point, move_distance = (0,0), boardsize=21):
    # returns the destination cell for a move distance tuple orderd in terms of
    # (move_west2east, move_north2south) so that a value of (1,3) moves on cell east and 3 cells
    # south and a value of (-3,-2) represents the cell that is 3 cells west and 2 cells to the north
    return ((start_point + move_distance[0]) % boardsize + 
            ((start_point//boardsize + move_distance[1])%boardsize) * boardsize)

In [4]:
def next_location(start_point,ship_action):
    # returns the destination cell for a ship that submits the ship_action
    if isinstance(ship_action,str):
        if ship_action.upper() == 'NORTH' or ship_action.upper() == 'N':
            next_cell = destination_cell(start_point, (0,-1))
        elif ship_action.upper() == 'SOUTH' or ship_action.upper() == 'S':
            next_cell = destination_cell(start_point, (0,1))
        elif ship_action.upper() == 'EAST' or ship_action.upper() == 'E':
            next_cell = destination_cell(start_point, (1,0))
        elif ship_action.upper() == 'WEST' or ship_action.upper() == 'W':
            next_cell = destination_cell(start_point, (-1,0))
        else:
            next_cell = start_point
    else:
        next_cell = start_point
    return next_cell

In [5]:
def infer_ship_action(start_point,end_point, cardinal_distance_list=cardinal_distance_list):
    # returns the ship action that would take it from start_point to end_point
    cell_change = cardinal_distance_list[start_point][end_point]
    if cell_change[0] == 0 and cell_change[1] == -1:
        ship_action = 'NORTH'
    elif cell_change[0] == 0 and cell_change[1] == 1:
        ship_action = 'SOUTH'
    elif cell_change[0] == 1 and cell_change[1] == 0:
        ship_action = 'EAST'
    elif cell_change[0] == -1 and cell_change[1] == 0:
        ship_action = 'WEST'
    elif cell_change[0] == 0 and cell_change[1] == 0:
        ship_action = None
    else:
        raise Exception('move not possible')
    return ship_action
    

In [6]:
def cells_in_path(start_point, end_point, boardsize=21):
    # Output is a list of points that are on the shortest path between 
    # the start point and end point including the start and end points. 
    # All points are represented as integers, so the
    # point on the northwest corner has value 0 and the point two steps south
    # and two steps east has value 44. The function cells_in_path(0,66) outputs the 
    # nine points [0,1,2,21,22,23,42,43,44] on the path between them.
    west2east, north2south = cardinal_distance_list[start_point][end_point]
    if west2east<0:
        eaststep = -1
    else:
        eaststep = 1
    
    if north2south<0:
        southstep = -1
    else:
        southstep = 1
        
    startrowbase = boardsize*(start_point // boardsize)
    allrows = [rowval%boardsize for rowval in 
               range(start_point//boardsize,
                     (1 + start_point//boardsize + north2south),
                     southstep)]
    allcols = [colval%boardsize for colval in 
               range(start_point%boardsize,
                     (1 + start_point%boardsize + west2east),
                     eaststep)]
    path_cells = []
    for row in allrows:
        for col in allcols:
            path_cells.append(boardsize*row + col)
    return path_cells
    

In [7]:
def cells_in_distance(location, max_distance=1):
    # returns the index references for all cells within the specific distance
    celllist = [location]
    if max_distance>0:
        # for each distance from 1 to max_distance, find all cells
        for dist in range(1,1+max_distance):
            # rotate around all the combinations of west-east and north-wouth adjustments that equal distance=dist
            west2east = np.concatenate( (np.arange(0,dist,step=1),np.arange(dist,-dist,step=-1),np.arange(-dist,0,step=1)))
            north2south = np.concatenate( (np.arange(-dist,dist,step=1),np.arange(dist,-dist,step=-1)))
            for idx in range(len(west2east)):
                celllist.append(destination_cell(location,( west2east[idx],north2south[idx])))
    return celllist

In [8]:
def distance_kernel(maxdist):
    # Creates a matrix of ones where the distance from center is less than
    # or equal to maxdist 
    # e.g. distance_kernel(2) = [[0,1,0],[1,1,1],[0,1,0]]
    kernelmat = np.zeros((1+2*maxdist,1+2*maxdist))
    for i in range(1+2*maxdist):
        for j in range(1+2*maxdist):
            if abs(i-maxdist)+abs(j-maxdist)<=maxdist:
                kernelmat[i,j] = 1
    return kernelmat

def halite_sum_matrix(halite_data, maxdist = 0, boardsize = 21):
    # Creates a matrix from the stepdata representing the board
    # where each cell has the sum of all halite within maxdist.
    # The halite data is the raw format =(stepdata[0]['observation']['halite'])
    # halite_sum_matrix(stepdata, maxdist = 0, boardsize = 21)
    halite_matrix = np.reshape(halite_data,(boardsize,boardsize))
    return signal.convolve2d(halite_matrix, 
                             distance_kernel(maxdist), 
                             mode='same', boundary='wrap', fillvalue=0)

def halite_sum_array(halite_data, maxdist = 0, boardsize = 21):
    return np.reshape(halite_sum_matrix(halite_data, maxdist, boardsize),(boardsize*boardsize))

In [9]:
def halite_total_change(new_obs,old_obs):
    halite_changes = {}
    for team_num in range(4):
        for ship_id in list(new_obs['players'][team_num][2].keys()):
            if ship_id in list(old_obs['players'][team_num][2].keys()):
                halite_changes[ship_id] = (new_obs['players'][team_num][2][ship_id][1]
                                                     -old_obs['players'][team_num][2][ship_id][1] )
    return halite_changes

In [10]:
def last_mining_yield(prev_action,old_obs):
    halite_mined = {}
    for team_num in range(4):
        for ship_id in list(prev_action[team_num].keys()):
            if prev_action[team_num][ship_id] is None:
                ship_location = old_obs['players'][team_num][2][ship_id][0]
                halite_mined[ship_id] = int(0.25*old_obs['halite'][ship_location])
            else:
                halite_mined[ship_id] = 0
    
    return halite_mined

In [11]:
def halite_delivery_yield(new_obs,old_obs):
    halite_delivery = [{},{},{},{}]
    for team_num in range(4):
        ## EXPENSES
        # new ships that are still alive after spawning
        num_new_living_ships = len([ship_id for ship_id in list(new_obs['players'][team_num][2].keys()) 
                             if ship_id not in list(old_obs['players'][team_num][2].keys())])
        
        # new ships that died immediately after spawning in the protection of the home base
        #new_dead_ships = 
        
        # new shipyards that are still living
        
        # new shipyards that died immediately after spawn
        
        ## INCOME
        # halite delivered by ships
        
        
    return halite_delivery
    

In [12]:
def present_prev_actions(ships_present,new_obs,old_obs):
    # infer each ship and shipyard action from the observation
    prev_action = [{},{},{},{}]

    for team_num in range(4):
        # Consider ships that still exist next turn
        for ship_id in list(new_obs['players'][team_num][2].keys()):
            if ship_id not in list(old_obs['players'][team_num][2].keys()):
                # 1) if didn't exist last turn, it was just spawned
                for shipyard_id in list(old_obs['players'][team_num][1].keys()):
                    if old_obs['players'][team_num][1][shipyard_id] == new_obs['players'][team_num][2][ship_id][0]:
                        prev_action[team_num][shipyard_id] = 'SPAWN'
            else:
                # 2) and if it did exist last turn, infer the action by its new location
                ship_move = infer_ship_action(start_point = old_obs['players'][team_num][2][ship_id][0],
                                  end_point = new_obs['players'][team_num][2][ship_id][0], 
                                  cardinal_distance_list=cardinal_distance_list)
                prev_action[team_num][ship_id] = ship_move
                        
    return prev_action
    

In [47]:
def investigate_mia(missing_ship_ids, new_obs, old_obs, verbose = False):
    # determines the last action and cause of death for ships that have gone missing
    
    # output variables
    missing_ship_last_action = {}
    missing_ship_killer_team = {}
    missing_ship_killer_uid = {}
    
    if len(missing_ship_ids)>0:
        
    
        # create list of missing ship team numbers, locations, and halite
        missing_ship_locations = {}
        missing_ship_halite = {}
        missing_ship_team_num = {}
        for ship_id in missing_ship_ids:
            for team_num in range(4):
                if ship_id in list(old_obs['players'][team_num][2].keys()):
                    missing_ship_team_num[ship_id] = team_num
                    missing_ship_locations[ship_id] = old_obs['players'][team_num][2][ship_id][0]
                    missing_ship_halite[ship_id] = old_obs['players'][team_num][2][ship_id][1]
        
        
        # infer the previous action for ships still present
        prev_action = present_prev_actions(list(new_obs['players'][team_num][2].keys()),new_obs,old_obs)
        
        # calculate the changes in halite for each ship
        halite_changes  = halite_total_change(new_obs,old_obs)
        halite_mined = last_mining_yield(prev_action,old_obs)
        for new_ship in new_obs['players'][team_num][2].keys():
            if new_ship not in halite_changes:
                halite_changes[new_ship] = 0
            if new_ship not in halite_mined:
                halite_mined[new_ship] = 0
        
        # deduce the last action, killer team and killer unit id for each missing ship among potential possibilities
        for ship_id in missing_ship_ids:
            potential_last_actions = []
            potential_killer_team = []
            potential_killer_uid = []

            adjacent_cells = [next_location(missing_ship_locations[ship_id],action_option) 
                              for action_option in ['N','S','E','W',None]]
            
            # for each adjacent cell, check if potential cause of death:
            #    - destroyed enemy shipyard if adjacent shipyard dissappeared (and it wasn't from another ship's attack)
            #    - was destroyed by an enemy ship if there is an enemy ship who now has its halite value
            #    - collided with enemy ship of equal halite value if adjacent enemy ship of same halite dissappeared
            #    - collided with friendly ship if ships of the same team dissappeared and were not killed by other means
            
            
            # converted to shipyard
            for team_num in range(4):
                for shipyard_id in list(new_obs['players'][team_num][1].keys()):
                    if shipyard_id not in list(old_obs['players'][team_num][1].keys()):
                        if (old_obs['players'][missing_ship_team_num[ship_id]][2][ship_id][0] 
                            == new_obs['players'][team_num][1][shipyard_id]):

                            potential_last_actions.append('CONVERT')
                            potential_killer_team.append(missing_ship_team_num[ship_id])
                            potential_killer_uid.append(ship_id)  # call it a suicide mission
                            # note if killer_uid is same as ship, it was convert to shipyard

            # **** Halite values visible in "obs" may not include the halite update from destroyed ships ****
            # destroyed by another ship who still exists and now has its halite value
            #for team_num in range(4):
                for other_ship in list(new_obs['players'][team_num][2].keys()):
                    try:
                        if ( (ship_id != other_ship) and                                                       # different ship 
                             (new_obs['players'][team_num][2][other_ship][0] in adjacent_cells) and           # in adjacent cell
                             (missing_ship_halite[ship_id] > 0) and                                                       # dead ship was not zero halite
                             ((halite_changes[other_ship] - halite_mined[other_ship]) == missing_ship_halite[ship_id])):  # halite increase = dead ship cargo

                            potential_last_actions.append(infer_ship_action(start_point=missing_ship_locations[ship_id],
                                                                            end_point=new_obs['players'][team_num][2][other_ship][0]))

                            potential_killer_team.append(team_num)
                            potential_killer_uid.append(other_ship)
                            # note this could be same team or different team, but killer_uid not the same as ship_id
                    except:
                        pass

            # destroyed by another ship who had same halite value and also died
            if len(missing_ship_ids)>1:
                for other_ship in missing_ship_ids:
                    if ( (other_ship is not ship_id) and
                         (missing_ship_halite[ship_id] == missing_ship_halite[other_ship]) and
                         (total_distance_list[missing_ship_locations[ship_id]][missing_ship_locations[other_ship]] < 3) ):
                            
                        other_adjacent_cells = [next_location(missing_ship_locations[other_ship],action_option) 
                              for action_option in ['N','S','E','W',None]]
                        collision_cells = [adjacent_cell for adjacent_cell in adjacent_cells if adjacent_cell in other_adjacent_cells]
                        ##### SMALL ERROR -- NO LOGIC TO DETERMINE WHICH COLLISION CELLS ARE VALID CHOICES, JUST CHOOSING FIRST OPTION
                        collision_cell = collision_cells[0]
                                                                                                          
                        potential_last_actions.append(infer_ship_action(start_point=missing_ship_locations[ship_id],
                                                                        end_point=collision_cell))
                        potential_killer_team.append(missing_ship_team_num[other_ship])
                        potential_killer_uid.append(other_ship)
                        # note this could be same team or different team, but killer_uid not the same as ship_id
                        
                        
            # destroyed enemy shipyard and was itself destroyed
            for team_num in range(4):
                if team_num != missing_ship_team_num[ship_id]:
                    for shipyard in list(old_obs['players'][team_num][1].keys()):
                        if (shipyard not in list(new_obs['players'][team_num][1].keys())) and (old_obs['players'][team_num][1][shipyard] in adjacent_cells):
                            potential_last_actions.append(infer_ship_action(start_point=missing_ship_locations[ship_id],
                                                                            end_point=old_obs['players'][team_num][1][shipyard]))
                            potential_killer_team.append(-1)      # -1 to signal shipyard kill
                            potential_killer_uid.append(shipyard)  # assign death to shipyard destroyed
                            # Note if team = -1, then shipyard destroyed

                            
                            
            # for the case where a ship is created in a shipyard and then immediately destroyed in battle
            for team_num in range(4):
                if team_num != missing_ship_team_num[ship_id]:
                    for shipyard_id in list(new_obs['players'][team_num][1].keys()):
                        if (total_distance_list[missing_ship_locations[ship_id]][new_obs['players'][team_num][1][shipyard_id]] == 1):
                            # assume that the ship attacked shipyard and was immediately killed by a newly spawned ship
                            potential_last_actions.append(infer_ship_action(start_point=missing_ship_locations[ship_id],
                                                                            end_point=new_obs['players'][team_num][1][shipyard_id]))
                            potential_killer_team.append(team_num)
                            potential_killer_uid.append(shipyard_id)  # we use the shipyard since we don't know the ship id

                            
            # destroyed by another ship with less halite 
            # potential error: change in in halite may not match ship cargo, and halite comparison lags step
            for team_num in range(4):
                for other_ship in list(new_obs['players'][team_num][2].keys()):

                    if ( (ship_id != other_ship) and                                                 # different ship
                         (new_obs['players'][team_num][2][other_ship][1] < old_obs['players'][missing_ship_team_num[ship_id]][2][ship_id][1]) and
                         (new_obs['players'][team_num][2][other_ship][0] in adjacent_cells)):       # in adjacent cell
                        
                        
                        potential_last_actions.append(infer_ship_action(start_point=missing_ship_locations[ship_id],
                                                                        end_point=new_obs['players'][team_num][2][other_ship][0] ))
                        potential_killer_team.append(team_num)
                        potential_killer_uid.append(other_ship)
                        # note this could be same team or different team, but killer_uid not the same as ship_id
                        
                        
            
            # for the case where a ship converts to a shipyard before being killed
            if len(missing_ship_ids)>1:
                for other_ship in missing_ship_ids:
                    if ( (other_ship is not ship_id) and
                         (total_distance_list[missing_ship_locations[ship_id]][missing_ship_locations[other_ship]] == 1) ):
                        
                        if missing_ship_halite[ship_id] > missing_ship_halite[other_ship]: # missing_ship converted
                            collision_cell = old_obs['players'][missing_ship_team_num[ship_id]][2][ship_id][0]
                            potential_last_actions.append('CONVERT')
                            potential_killer_team.append(missing_ship_team_num[ship_id])
                            potential_killer_uid.append(ship_id)
                        else:  # other ship converted
                            collision_cell = old_obs['players'][missing_ship_team_num[other_ship]][2][other_ship][0]
                            potential_last_actions.append(infer_ship_action(start_point=missing_ship_locations[ship_id],
                                                                            end_point=collision_cell))
                            potential_killer_team.append(missing_ship_team_num[other_ship]-10)
                            potential_killer_uid.append(missing_ship_team_num[other_ship])
                            
            # do a version that does not check halite to account for halite counting errors in mining or concurrent battles
            for team_num in range(4):
                for other_ship in list(new_obs['players'][team_num][2].keys()):
                    if ( (ship_id != other_ship) and                                                 # different ship
                         (new_obs['players'][team_num][2][other_ship][0] in adjacent_cells)):       # in adjacent cell
                                                      
                        potential_last_actions.append(infer_ship_action(start_point=missing_ship_locations[ship_id],
                                                                        end_point=new_obs['players'][team_num][2][other_ship][0] ))
                        potential_killer_team.append(team_num)
                        potential_killer_uid.append(other_ship)
                        # note this could be same team or different team, but killer_uid not the same as ship_id
                        
            
            
            # When there are multiple potential options, select the first (highest priority)
            missing_ship_last_action[ship_id] = potential_last_actions[0]
            missing_ship_killer_team[ship_id] = potential_killer_team[0]
            missing_ship_killer_uid[ship_id] = potential_killer_uid[0]
            
    if verbose:
        if len(missing_ship_ids)>0:
            for ship_id in missing_ship_ids:
 
                if missing_ship_last_action[ship_id] is None:
                    print_action = 'MINE'
                else:
                    print_action = missing_ship_last_action[ship_id]
                print(ship_id + ' at cell ' + str(missing_ship_locations[ship_id]) + 
                      ' chose action ' + print_action + 
                      ' to cell ' + str(next_location(missing_ship_locations[ship_id],missing_ship_last_action[ship_id])) +
                      ' and was killed by ' + missing_ship_killer_uid[ship_id] +
                      ' from team ' + str(missing_ship_killer_team[ship_id]) )
    
    return missing_ship_last_action, missing_ship_killer_team, missing_ship_killer_uid                                                                                          

In [48]:
def infer_previous_action(new_obs, old_obs, verbose = False):
    # infer previous actions from the observations
    
    present_ship_ids = []
    missing_ship_ids = []
    for team_num in range(4):
        present_ship_ids += list(new_obs['players'][team_num][2].keys())
        missing_ship_ids += [ship_id for ship_id in list(old_obs['players'][team_num][2].keys()) if ship_id not in present_ship_ids]
    
    prev_actions = present_prev_actions(present_ship_ids,new_obs,old_obs)
    missing_ship_last_action, _, _ = investigate_mia(missing_ship_ids, new_obs, old_obs, verbose = verbose)
    
    for team_num in range(4):
        prev_actions[team_num].update({missing_ship_id:missing_ship_last_action[missing_ship_id] 
                                      for missing_ship_id in list(old_obs['players'][team_num][2].keys()) 
                                       if missing_ship_id not in list(new_obs['players'][team_num][2].keys())}) 
    return prev_actions

In [49]:
def nearby_features(ship_id, team, obs, max_distance=3):
    
    # outputs the pattern value for the distance around the ship
    ship_location = obs['players'][team][2][ship_id][0]
    ship_halite = obs['players'][team][2][ship_id][1]
    nearby_cells = cells_in_distance(ship_location,max_distance)
    nearby_features_array = np.zeros(shape=len(nearby_cells),dtype=np.uint64)
    
    # 1 - Friendly shipyard
    for shipyard_location in list(obs['players'][team][1].values()):
        if shipyard_location in nearby_cells:
            nearby_features_array[nearby_cells.index(shipyard_location)] = 1
        
    # 2 - Friendly ships
    friendly_ship_locations = [shipdata[0] for shipdata in list(obs['players'][team][2].values())]
    for friendly_ship_location in friendly_ship_locations:
        if friendly_ship_location in nearby_cells and friendly_ship_location is not ship_location:
            nearby_features_array[nearby_cells.index(friendly_ship_location)] = 2
    
    
    for other_team in range(4):
        if other_team is not team:
    
            # 3 - Enemy shipyards
            for shipyard_location in list(obs['players'][other_team][1].values()):
                if shipyard_location in nearby_cells:
                    nearby_features_array[nearby_cells.index(shipyard_location)] = 3
                    
            # 4 - Enemy fat ships
            enemy_fleet_data = list(obs['players'][other_team][2].values())
            for enemy_ship_data in enemy_fleet_data:
                if (enemy_ship_data[0] in nearby_cells) and (enemy_ship_data[1] > ship_halite):
                    nearby_features_array[nearby_cells.index(enemy_ship_data[0])] = 4
                    
            # 5 - Enemy lean ships
            for enemy_ship_data in enemy_fleet_data:
                if (enemy_ship_data[0] in nearby_cells) and (enemy_ship_data[1] <= ship_halite):
                    nearby_features_array[nearby_cells.index(enemy_ship_data[0])] = 5
                    
    return nearby_features_array

In [50]:
def rotational_equivalents(nearby_features_array):
    # returns indices for "nearby_features" indices that are strategically equivalent, 
    # simply rotated or reflected game boards
    # order of manipulations is: rotation counter-clockwise once, twice, three-times, 
    # mirror left/right, mirror up/down, mirror nortwest/southest, mirror northeast/southwest
    
    rotations = np.array([[0,0,0,0,0,0,0,0],
                            [1,2,3,4,1,3,2,4],
                            [2,3,4,1,4,2,1,3],
                            [3,4,1,2,3,1,4,2],
                            [4,1,2,3,2,4,3,1],
                            [5,7,9,11,5,9,7,11],
                            [6,8,10,12,12,8,6,10],
                            [7,9,11,5,11,7,5,9],
                            [8,10,12,6,10,6,12,8],
                            [9,11,5,7,9,5,11,7],
                            [10,12,6,8,8,12,10,6],
                            [11,5,7,9,7,11,9,5],
                            [12,6,8,10,6,10,8,12],
                            [13,16,19,22,13,19,16,22],
                            [14,17,20,23,24,18,15,21],
                            [15,18,21,24,23,17,14,20],
                            [16,19,22,13,22,16,13,19],
                            [17,20,23,14,21,15,24,18],
                            [18,21,24,15,20,14,23,17],
                            [19,22,13,16,19,13,22,16],
                            [20,23,14,17,18,24,21,15],
                            [21,24,15,18,17,23,20,14],
                            [22,13,16,19,16,22,19,13],
                            [23,14,17,20,15,21,18,24],
                            [24,15,18,21,14,20,17,23]])
    
    rotational_indices = []
    for r in range(8):
        rotational_indices.append( [nearby_features_array[rotations[idx,r]] for idx in range(25)] )
    
    return rotational_indices

In [51]:
# transform the nearby cells into a single integer
def nearby_feature_code(nearby_features_array):
    return np.sum( np.multiply(nearby_features_array[1:],     # skip first element which contains the ship itself
                               np.power(6, np.arange(0,24,dtype=np.uint64))))

In [52]:
class BoardOutput:
    def __init__(self, new_obs):
        self.board_size = 21
        self.board_halite_total = int(sum(new_obs['halite']))
        self.board_ship_total = sum([len(new_obs['players'][team_num][2]) for team_num in range(4)])
        self.board_shipyard_total = sum([len(new_obs['players'][team_num][1]) for team_num in range(4)])
        

        #self.board_asset_map = [[-1,-1] for cellnum in range(self.board_size**2)]
        self.board_ship_map = [-1 for cellnum in range(self.board_size**2)]
        self.board_shipyard_map = [-1 for cellnum in range(self.board_size**2)]
        for team_num in range(4):
            for shipyard_data in list(new_obs['players'][team_num][1].values()):
                #self.board_asset_map[shipyard_data][0] = team_num
                self.board_shipyard_map[shipyard_data] = team_num
            for ship_data in list(new_obs['players'][team_num][2].values()):
                #self.board_asset_map[ship_data[0]][1] = team_num
                self.board_ship_map[ship_data[0]] = team_num
        
    def __repr__(self):
        outstring = 'class:BoardOutput \n'
        for key,value in self.__dict__.items():
            if key == 'board_halite_at_distance':
                outstring += key + ':' + str(len(value)) + ' arrays \n'
            elif key == 'board_ship_map':
                outstring += key + ':' + str(self.board_ship_total) + ' ships on map \n'
            elif key == 'board_shipyard_map':
                outstring += key + ':' + str(self.board_shipyard_total) + ' shipyards on map \n'
            else:
                outstring += key + ':' + str(value) + '\n'
        return outstring

In [53]:
class TeamOutput:
    def __init__(self, new_obs):
        self.team_list = [0,1,2,3]
        self.team_halite = [new_obs['players'][team_num][0] for team_num in range(4)]
        self.team_cargo = [sum([shipdata[1] for shipdata in new_obs['players'][team_num][2].values()]) for team_num in range(4)]
        self.team_shipyards = [len(new_obs['players'][team_num][1]) for team_num in range(4)]
        self.team_ships = [len(new_obs['players'][team_num][2]) for team_num in range(4)]
        
    def __repr__(self):
        outstring = 'class:TeamOutput \n'
        for key,value in self.__dict__.items():
            outstring += key + ':' + str(value) + '\n'
        return outstring

In [54]:
class ShipOutput:
    def __init__(self, new_obs, include_future = False):
        
        ### Initialize Variables
        self.ship_list = []

        self.ship_team_num = []
 
        self.ship_cargo = []
        self.ship_dist_shipyard = []
        self.ship_dist_enemy_yard = []
        self.ship_halite_d0 = []
        self.ship_halite_d1 = []
        self.ship_halite_d2 = []
        self.ship_halite_d3 = []
        self.ship_halite_d4 = []
        self.ship_halite_d5 = []
        self.ship_halite_d6 = []

        self.ship_friendly_d1 = []
        self.ship_friendly_d2 = []
        self.ship_friendly_d3 = []
        self.ship_friendly_d4 = []

        self.ship_fat_enemy_d1 = []
        self.ship_fat_enemy_d2 = []
        self.ship_fat_enemy_d3 = []
        self.ship_fat_enemy_d4 = []

        self.ship_lean_enemy_d1 = []
        self.ship_lean_enemy_d2 = []
        self.ship_lean_enemy_d3 = []
        self.ship_lean_enemy_d4 = []
        self.ship_lean_blocking = []

        self.ship_strong_seige = []
        self.ship_weak_seige = []
        self.ship_defend_seige = []

        self.ship_pattern = []
        
       
        if include_future:

            self.Future_next_action = []
            self.Future_life = []
            self.Future_halite_delivered = []
            self.Future_ship_kills = []
            self.Future_shipyard_kill = []
            self.Future_shipyard_defend = []
            self.Future_first_convert = []
            self.Future_later_convert = []

        ### Calculate Values
        
        # Calculate sum of halite for various distances relative to each board cell
        board_halite_at_distance = []
        for d in range(7):
            board_halite_at_distance.append( halite_sum_array(new_obs['halite'], maxdist = d) )
        
        # Pull data for each team, calculating locations of friendly and enemy units
        for team_num,team_data in enumerate(new_obs['players']):

            # make a list of friendly and enemy shipyard locations
            friendly_shipyard_locations = list(team_data[1].values())

            enemy_shipyard_locations = []
            for enemy_num in range(4):
                if enemy_num is not team_num:
                    enemy_shipyard_locations += list(new_obs['players'][enemy_num][1].values())

            # make lists of friendly and enemy ships
            friendly_ship_locations = [shipdata[0] for shipdata in list(team_data[2].values())]
            friendly_ship_array = [1 if cellidx in friendly_ship_locations else 0 for cellidx in range(21**2)]

            enemy_ship_locations = []
            enemy_ship_cargo = []
            enemy_ship_data = []
            for enemy_num in range(4):
                if enemy_num is not team_num:
                    for shipdata in list(new_obs['players'][enemy_num][2].values()):
                        enemy_ship_data.append(shipdata)
                        enemy_ship_locations.append(shipdata[0])
                        enemy_ship_cargo.append(shipdata[1])


            # SHIP-LEVEL DATA
            for ship_name in list(team_data[2].keys()):
                self.ship_list.append(ship_name)
                self.ship_team_num.append(team_num)               
                self.ship_cargo.append(team_data[2][ship_name][1])
                
                
                # Calculate distances to units
                ship_location = team_data[2][ship_name][0]

                if len(friendly_shipyard_locations)<1:
                    friendly_shipyard_distances = [21]
                    closest_shipyard = None
                else:
                    friendly_shipyard_distances = [total_distance_list[ship_location][shipyards] 
                                                   for shipyards in friendly_shipyard_locations]
                    closest_shipyard = friendly_shipyard_locations[friendly_shipyard_distances.index(min(friendly_shipyard_distances))]


                if len(enemy_shipyard_locations)<1:
                    enemy_shipyard_distances = [21]
                else:
                    enemy_shipyard_distances = [total_distance_list[ship_location][shipyards] 
                             for shipyards in enemy_shipyard_locations]

                # distance to friendly ships
                friendly_ship_distances = [total_distance_list[ship_location][friendly_location] 
                                           for friendly_location in friendly_ship_locations]

                # distance to enemies with more halite
                fat_enemy_locations = [shipdata[0] for shipdata in enemy_ship_data if shipdata[1]>team_data[2][ship_name][1] ]
                if len(fat_enemy_locations)<1:
                    fat_enemy_distances = [21]
                else:
                    fat_enemy_distances = [total_distance_list[ship_location][fat_enemy_location] 
                                           for fat_enemy_location in fat_enemy_locations]

                # distance to enemy ships with less halite
                lean_enemy_locations = [shipdata[0] for shipdata in enemy_ship_data if shipdata[1]<=team_data[2][ship_name][1] ]
                if len(lean_enemy_locations)<1:
                    lean_enemy_distances = [21]
                else:
                    lean_enemy_distances = [total_distance_list[ship_location][lean_enemy_location]
                                           for lean_enemy_location in lean_enemy_locations]


                # distance to enemy and friendly shipyards
                self.ship_dist_shipyard.append( min(friendly_shipyard_distances) )    
                self.ship_dist_enemy_yard.append(min(enemy_shipyard_distances) )

                # amount of halite on the board at distance 0,1,2,3,4,5,6
                self.ship_halite_d0.append(int(board_halite_at_distance[0][ship_location]))
                self.ship_halite_d1.append(int(board_halite_at_distance[1][ship_location]))
                self.ship_halite_d2.append(int(board_halite_at_distance[2][ship_location]))
                self.ship_halite_d3.append(int(board_halite_at_distance[3][ship_location]))
                self.ship_halite_d4.append(int(board_halite_at_distance[4][ship_location]))
                self.ship_halite_d5.append(int(board_halite_at_distance[5][ship_location]))
                self.ship_halite_d6.append(int(board_halite_at_distance[6][ship_location]))

                # amount of friendly ships on board within distance 1,2,3,4
                self.ship_friendly_d1.append( sum([(friendly_ship_distance<=1) 
                                                   for friendly_ship_distance in friendly_ship_distances])-1)
                self.ship_friendly_d2.append( sum([(friendly_ship_distance<=2) 
                                                   for friendly_ship_distance in friendly_ship_distances])-1)
                self.ship_friendly_d3.append( sum([(friendly_ship_distance<=3) 
                                                   for friendly_ship_distance in friendly_ship_distances])-1)
                self.ship_friendly_d4.append( sum([(friendly_ship_distance<=4) 
                                                   for friendly_ship_distance in friendly_ship_distances])-1)


                # fat enemy ships present at distance 1,2,3

                self.ship_fat_enemy_d1.append( sum([(fat_enemy_distance<=1) 
                                                   for fat_enemy_distance in fat_enemy_distances]))
                self.ship_fat_enemy_d2.append( sum([(fat_enemy_distance<=2) 
                                                   for fat_enemy_distance in fat_enemy_distances]))
                self.ship_fat_enemy_d3.append( sum([(fat_enemy_distance<=3) 
                                                   for fat_enemy_distance in fat_enemy_distances]))
                self.ship_fat_enemy_d4.append( sum([(fat_enemy_distance<=4) 
                                                   for fat_enemy_distance in fat_enemy_distances]))


                # fat enemy ships present at distance 1,2,3

                self.ship_lean_enemy_d1.append( sum([(lean_enemy_distance<=1) 
                                                   for lean_enemy_distance in lean_enemy_distances]))
                self.ship_lean_enemy_d2.append( sum([(lean_enemy_distance<=2) 
                                                   for lean_enemy_distance in lean_enemy_distances]))
                self.ship_lean_enemy_d3.append( sum([(lean_enemy_distance<=3) 
                                                   for lean_enemy_distance in lean_enemy_distances]))
                self.ship_lean_enemy_d4.append( sum([(lean_enemy_distance<=4) 
                                                   for lean_enemy_distance in lean_enemy_distances]))

                if closest_shipyard is None:
                    self.ship_lean_blocking.append(0)
                else:
                    shipyard_path_cells = cells_in_path(ship_location, closest_shipyard)
                    self.ship_lean_blocking.append( sum([lean_enemy_location in shipyard_path_cells 
                                                         for lean_enemy_location in lean_enemy_locations]))

                
                # SEIGE ANALYTICS
                # Strong seige position: within distance one of enemy shipyard and that enemy has <500 halite and no lean ship within one move of shipyard
                # Weak seige position: within distance one of enemy shipyard but that enemy has >=500 halite or a lean ship within one move of shipyard

                # if participating in enemy seige
                if min(enemy_shipyard_distances ) == 1:
                    besieged_shipyard_location = enemy_shipyard_locations[enemy_shipyard_distances.index(1)]
                    # initialize beseiged team halite and lean defense distances
                    beseiged_team_halite = 0
                    lean_defense_distances = [0]
                    # find the besieged team halite and lean defense distances
                    for enemy_num in range(4):
                        if enemy_num is not team_num:
                            if besieged_shipyard_location in list(new_obs['players'][enemy_num][1].values()):
                                beseiged_team_halite = new_obs['players'][enemy_num][0]
                                lean_defense_distances = [total_distance_list[besieged_shipyard_location][lean_enemy_location]
                                           for lean_enemy_location in lean_enemy_locations]

                    if beseiged_team_halite<500 and min(lean_defense_distances)>1:
                        ship_strong_seige = 1
                        ship_weak_seige = 0
                    else:
                        ship_strong_seige = 0
                        ship_weak_seige = 1
                else:
                    ship_strong_seige = 0
                    ship_weak_seige = 0

                # if ship is in position to defend against an enemy seige
                ship_defend_seige = 0
                if min(friendly_shipyard_distances) <=1:
                    # close enough to protect against enemy seige
                    nearby_shipyard_location = friendly_shipyard_locations[friendly_shipyard_distances.index(min(friendly_shipyard_distances))]
                    beatable_enemy_locations = [shipdata[0] for shipdata in enemy_ship_data if shipdata[1]>=team_data[2][ship_name][1] ]
                    beatable_enemy_distances = [total_distance_list[nearby_shipyard_location][beatable_enemy_location]
                                           for beatable_enemy_location in beatable_enemy_locations]
                    if len(beatable_enemy_distances)>0:
                        if min(beatable_enemy_distances) == 1:
                            ship_defend_seige = 1

                self.ship_strong_seige.append(ship_strong_seige)
                self.ship_weak_seige.append(ship_weak_seige)
                self.ship_defend_seige.append(ship_defend_seige)

                
                # pattern of ships within distance of 4 moves
                self.ship_pattern.append(
                    nearby_feature_code(nearby_features(ship_name, team_num, new_obs)))
                
                if include_future:
                    # initialize the future variables
                    self.Future_next_action.append('X')
                    self.Future_life.append(0)
                    self.Future_halite_delivered.append(0)
                    self.Future_ship_kills.append(0)
                    self.Future_shipyard_kill.append(0)
                    self.Future_shipyard_defend.append(0)
                    self.Future_first_convert.append(0)
                    self.Future_later_convert.append(0)

    def __repr__(self):
        outstring = 'class:ShipOutput \n'
        if len(self.ship_list) < 12:
            for key,value in self.__dict__.items():
                outstring += key + ': ' + str(value) + '\n'
        else:
            for key,value in self.__dict__.items():
                outstring += key + ': ' + str(len(value)) + ' values \n'
        return outstring

In [55]:
class CumulativeData:
    def __init__(self, new_obs):
        self.step = 0
        self.obs = new_obs
        self.old_obs = new_obs
        self.all_ship_names = [] # list of all past and preesnt ship names in game, index of this list is shipNumId
        self.all_ship_numbers = [] # list of all past and present ship names in game (aka shipNumId)
        self.all_ship_team_num = {} # dictionary of ship_names as keys and team numbers as values
        for team_num in range(4):
            for ship_name in list(new_obs['players'][team_num][2].keys()):
                self.all_ship_names.append(ship_name)
                self.all_ship_numbers.append(self.all_ship_names.index(ship_name))
                self.all_ship_team_num[ship_name] = team_num
        
        self.team_ship_converts = [0,0,0,0]   # ship converts to shipyard (gone)
        self.team_ship_collides = [0,0,0,0]   # ship collides with friendly (gone)
        self.team_ship_deaths = [0,0,0,0]     # ship destroyed by enemy (gone)
        self.team_shipyard_births = [0,0,0,0]     # ship destroyed by enemy (gone)
        self.team_shipyard_kills = [0,0,0,0]  # ship destroys enemy shipyard (gone)
        self.team_ship_births = [0,0,0,0]     # new ship created by friendly shipyard
        self.team_ship_kills = [0,0,0,0]      # enemy ship destroyed by team
        self.team_shipyard_deaths = [0,0,0,0] # friendly shipyard destroyed by enemy
        self.team_shipyard_saves = [0,0,0,0]  # saved a friendly shipyard from death
        
        self.team_actions_previous = [{},{},{},{}]
        self.team_actions_2steps_ago = [{},{},{},{}]
        self.team_actions_3steps_ago = [{},{},{},{}]
        
        
        
    def update(self, new_obs):
        for team_num in range(4):
            for ship_name in list(new_obs['players'][team_num][2].keys()):
                if ship_name not in self.all_ship_names:
                    self.all_ship_names.append(ship_name)
                    self.all_ship_numbers.append(self.all_ship_names.index(ship_name))
                    self.all_ship_team_num[ship_name] = team_num
                    
        team_actions = infer_previous_action(new_obs, self.obs, verbose = False)
        self.team_actions_3steps_ago = self.team_actions_2steps_ago
        self.team_actions_2steps_ago = self.team_actions_previous
        self.team_actions_previous = team_actions
        
        mia_ships = []
        for team_num in range(4):
            self.team_ship_births[team_num] += sum([ship_act == 'SPAWN' 
                                                    for ship_act in team_actions[team_num].values()])
            
            self.team_shipyard_births[team_num] += len([shipyard for shipyard 
                                                        in list(new_obs['players'][team_num][1].keys()) 
                                                        if shipyard not in list(self.obs['players'][team_num][1].keys())])
            
            self.team_shipyard_deaths[team_num] += len([shipyard for shipyard 
                                                        in list(self.obs['players'][team_num][1].keys()) 
                                                        if shipyard not in list(new_obs['players'][team_num][1].keys())])
            
            # list of missing ships
            mia_ships += [ship_id for ship_id in self.obs['players'][team_num][2] if ship_id not in new_obs['players'][team_num][2]]
            
        # determine cause of death and tally it up
        if len(mia_ships)>0:
            ms_action, ms_killer_team, ms_killer_uid  = investigate_mia(mia_ships, new_obs, self.obs, verbose = False)
            for ship_id in mia_ships:
                if ms_killer_team[ship_id] < 0:                 # killed a shipyard
                    self.team_shipyard_kills[self.all_ship_team_num[ship_id]] += 1
                    if ms_killer_team[ship_id] < -1:   # killed shipyard that was simultaneously created
                        self.team_ship_converts[ms_killer_team[ship_id]+10] += 1
                        self.team_shipyard_births[ms_killer_team[ship_id]+10] += 1
                        self.team_shipyard_deaths[ms_killer_team[ship_id]+10] += 1
                elif ms_killer_team[ship_id] == self.all_ship_team_num[ship_id]:         # killed by same team...
                    if ms_killer_uid[ship_id] == ship_id:
                        self.team_ship_converts[self.all_ship_team_num[ship_id]] += 1    # as a result of converting to shipyard
                    else:
                        self.team_ship_collides[self.all_ship_team_num[ship_id]] += 1         # as a result of a same-team collision
                else:
                    self.team_ship_deaths[self.all_ship_team_num[ship_id]] += 1
                    self.team_ship_kills[ms_killer_team[ship_id]] += 1
                    if ms_killer_uid[ship_id] in self.obs['players'][ms_killer_team[ship_id]][2]:
                        killer_location = next_location(start_point = self.obs['players'][ms_killer_team[ship_id]][2][ms_killer_uid[ship_id]][0],
                                                    ship_action = ms_action[ship_id])
                        # tally if the killer saved a shipyard
                        if (killer_location in list(new_obs['players'][ms_killer_team[ship_id]][1].values())):    
                            self.team_shipyard_saves[ms_killer_team[ship_id]] += 1 
                            
        self.old_obs = self.obs
        self.obs = new_obs
    
    def __repr__(self):
        outstring = 'class:CumulativeData \n'
        if len(self.all_ship_names)>12:
            for key,value in self.__dict__.items():
                if key == 'obs':
                    outstring += key + ': Halite game observation data for step ' +str(self.step) + ' \n'
                elif key == 'old_obs':
                    outstring += key + ': Halite game observation data for step ' +str(self.old_obs['step']) + ' \n'
                elif key == 'all_ship_names':
                    outstring += key + ': ' +str(len(value)) + ' ship uid values \n'
                elif key == 'all_ship_numbers':
                    outstring += key + ': ' +str(len(value)) + ' unique integer identifiers \n'
                elif key == 'all_ship_team_num':
                    outstring += key + ': dictionary of ' +str(len(value)) + ' ship names mapped to team numbers \n'
                elif key == 'team_actions_previous':
                    outstring += key + ': 4-element list of dictionaries of team ship names mapped to ship actions \n'
                elif key == 'team_actions_2steps_ago':
                    outstring += key + ': 4-element list of dictionaries of team ship names mapped to ship actions \n'
                elif key == 'team_actions_3steps_ago':
                    outstring += key + ': 4-element list of dictionaries of team ship names mapped to ship actions \n'  
                    
                else:
                    outstring += key + ':' + str(value) + '\n'
        else:
            for key,value in self.__dict__.items():
                if key == 'obs':
                    outstring += key + ': Halite game observation data for step ' +str(self.obs['step']) + ' \n'
                elif key == 'old_obs':
                    outstring += key + ': Halite game observation data for step ' +str(self.old_obs['step']) + ' \n'
                else:
                    outstring += key + ':' + str(value) + '\n'
        return outstring
    

In [56]:
def create_step_features(new_obs, cumulative_data, include_future = False):
    # Creates a dictionary of step features where each key is a column name and each value is an array with entries for every ship.
    # This is intended to be used to make an easy dataframe with all or some of the features

    step_num = new_obs['step']
    
    # Output data objects
    board_output = BoardOutput(new_obs)
    team_output = TeamOutput(new_obs)
    ship_output = ShipOutput(new_obs, include_future)
    
    # number of ships/observations for this step
    nobs = len(ship_output.ship_list)
    
    # create lists of actions
    action_previous = []
    action_2steps_ago = []
    action_3steps_ago = []
    
    for idx in range(len(ship_output.ship_list)):
        if ship_output.ship_list[idx] in cumulative_data.team_actions_previous[ship_output.ship_team_num[idx]]:
            action_previous.append(cumulative_data.team_actions_previous[ship_output.ship_team_num[idx]][ship_output.ship_list[idx]])
        else:
            action_previous.append(None)
        if ship_output.ship_list[idx] in cumulative_data.team_actions_2steps_ago[ship_output.ship_team_num[idx]]:
            action_2steps_ago.append(cumulative_data.team_actions_2steps_ago[ship_output.ship_team_num[idx]][ship_output.ship_list[idx]])
        else:
            action_2steps_ago.append(None)
        if ship_output.ship_list[idx] in cumulative_data.team_actions_3steps_ago[ship_output.ship_team_num[idx]]:
            action_3steps_ago.append(cumulative_data.team_actions_3steps_ago[ship_output.ship_team_num[idx]][ship_output.ship_list[idx]])
        else:
            action_3steps_ago.append(None)
    
    
    step_features = {'ship_id':                   ship_output.ship_list,
                     'step':                      [step_num]*nobs,
                     'team_num':                  ship_output.ship_team_num,
                     'board_halite_total':        [board_output.board_halite_total]*nobs,
                     'board_ship_total':          [board_output.board_ship_total]*nobs,
                     'board_shipyard_total':      [board_output.board_shipyard_total]*nobs,
                     'team_halite':               [team_output.team_halite[team_num] for team_num in ship_output.ship_team_num],
                     'team_cargo':                [team_output.team_cargo[team_num] for team_num in ship_output.ship_team_num],
                     'team_ships':                [team_output.team_ships[team_num] for team_num in ship_output.ship_team_num],
                     'team_ship_births':          [cumulative_data.team_ship_births[team_num] for team_num in ship_output.ship_team_num],
                     'team_ship_deaths':          [cumulative_data.team_ship_deaths[team_num] for team_num in ship_output.ship_team_num],
                     'team_ship_collisions':      [cumulative_data.team_ship_collides[team_num] for team_num in ship_output.ship_team_num],
                     'team_shipyards':            [team_output.team_shipyards[team_num] for team_num in ship_output.ship_team_num],
                     'team_shipyard_births':      [cumulative_data.team_shipyard_births[team_num] for team_num in ship_output.ship_team_num],
                     'team_shipyard_deaths':      [cumulative_data.team_shipyard_deaths[team_num] for team_num in ship_output.ship_team_num],
                     'team_shipyard_saves':       [cumulative_data.team_shipyard_saves[team_num] for team_num in ship_output.ship_team_num],
                     'team_enemy_ship_kills':     [cumulative_data.team_ship_kills[team_num] for team_num in ship_output.ship_team_num],
                     'team_enemy_shipyard_kills': [cumulative_data.team_shipyard_kills[team_num] for team_num in ship_output.ship_team_num],
                     'ship_cargo':                ship_output.ship_cargo,
                     'ship_dist_shipyard':        ship_output.ship_dist_shipyard,
                     'enemies_blocking_shipyard': ship_output.ship_lean_blocking,
                     'dist_enemy_shipyard':       ship_output.ship_dist_enemy_yard,
                     'halite_dist_0':             ship_output.ship_halite_d0,
                     'halite_dist_1':             ship_output.ship_halite_d1,
                     'halite_dist_2':             ship_output.ship_halite_d2,
                     'halite_dist_3':             ship_output.ship_halite_d3,
                     'halite_dist_4':             ship_output.ship_halite_d4,
                     'halite_dist_5':             ship_output.ship_halite_d5,
                     'halite_dist_6':             ship_output.ship_halite_d6,
                     'friendly_ships_dist_1':     ship_output.ship_friendly_d1,
                     'friendly_ships_dist_2':     ship_output.ship_friendly_d2,
                     'friendly_ships_dist_3':     ship_output.ship_friendly_d3,
                     'friendly_ships_dist_4':     ship_output.ship_friendly_d4,
                     'fat_enemy_ships_dist_1':    ship_output.ship_fat_enemy_d1,
                     'fat_enemy_ships_dist_2':    ship_output.ship_fat_enemy_d2,
                     'fat_enemy_ships_dist_3':    ship_output.ship_fat_enemy_d3,
                     'fat_enemy_ships_dist_4':    ship_output.ship_fat_enemy_d4,
                     'lean_enemy_ships_dist_1':   ship_output.ship_lean_enemy_d1,
                     'lean_enemy_ships_dist_2':   ship_output.ship_lean_enemy_d2,
                     'lean_enemy_ships_dist_3':   ship_output.ship_lean_enemy_d3,
                     'lean_enemy_ships_dist_4':   ship_output.ship_lean_enemy_d4,
                     'strong_seige':              ship_output.ship_strong_seige,
                     'weak_seige':                ship_output.ship_strong_seige,
                     'defend_seige':              ship_output.ship_strong_seige,
                     'tactical_pattern':          ship_output.ship_pattern,
                     'action_previous':           action_previous,
                     'action_2steps_ago':         action_2steps_ago,
                     'action_3steps_ago':         action_3steps_ago}
    
    if include_future:
        step_features.update({'Future_next_action':      ship_output.Future_next_action,
                              'Future_life':             ship_output.Future_life,
                              'Future_halite_delivered': ship_output.Future_halite_delivered,
                              'Future_ship_kills':       ship_output.Future_ship_kills,
                              'Future_shipyard_kill':    ship_output.Future_shipyard_kill,
                              'Future_shipyard_defend':  ship_output.Future_shipyard_defend,
                              'Future_first_convert':    ship_output.Future_first_convert,
                              'Future_later_convert':    ship_output.Future_later_convert})
    
    return step_features                

In [57]:
def game_df_update(game_df, step_df, cumulative_data):
    # considers the events of the most recent step, as recorded in cumulative_data, and adds them as future outcome data
    # to the past entries in game_df
    
    # NEXT ACTION
    team_actions = infer_previous_action(cumulative_data.obs, cumulative_data.old_obs, verbose = False)
    for team_num in range(4):
        for ship_id in list(team_actions[team_num].keys()):
            game_df.loc[(game_df['step'].eq(cumulative_data.old_obs['step']) & game_df['ship_id'].eq(ship_id)), 'Future_next_action'] = team_actions[team_num][ship_id]
    
    # FUTURE LIFE
    current_ships = []
    for team_num in range(4):
        current_ships += cumulative_data.obs['players'][team_num][2].keys()
    game_df.loc[game_df.ship_id.isin(current_ships),'Future_life'] = game_df.loc[game_df.ship_id.isin(current_ships),'Future_life']+1
    
    # CONVERT TO SHIPYARD
    converting_ships = []
    for team_num in range(4):
        converting_ships += [ship_id for ship_id in list(team_actions[team_num].keys()) if team_actions[team_num][ship_id] == 'CONVERT']
    if cumulative_data.obs['step'] == 1:
        for ship_id in converting_ships:
            game_df.loc[game_df.ship_id == ship_id,'Future_first_convert'] = 1
    else:
        for ship_id in converting_ships:
            game_df.loc[game_df.ship_id == ship_id,'Future_later_convert'] = 1
        
    # HALITE DELIVERED
    for team_num in range(4):
        shipyard_locations = list(cumulative_data.obs['players'][team_num][1].values())
        if len(shipyard_locations)>0:
            for ship_id in cumulative_data.old_obs['players'][team_num][2].keys():
                if (next_location(start_point=cumulative_data.old_obs['players'][team_num][2][ship_id][0],
                                  ship_action = team_actions[team_num][ship_id]) in shipyard_locations):
                    if team_actions[team_num][ship_id] == 'CONVERT':
                        if cumulative_data.old_obs['players'][team_num][2][ship_id][1] > 500:
                            game_df.loc[game_df.ship_id == ship_id,'Future_halite_delivered'] = (
                                game_df.loc[game_df.ship_id == ship_id,'Future_halite_delivered'] + 
                                   (cumulative_data.old_obs['players'][team_num][2][ship_id][1] - 500))
                    else:
                         game_df.loc[game_df.ship_id == ship_id,'Future_halite_delivered'] = (
                            game_df.loc[game_df.ship_id == ship_id,'Future_halite_delivered'] + 
                                (cumulative_data.old_obs['players'][team_num][2][ship_id][1]))
                            
    # SHIP AND SHIPYARD KILLS
    mia_ships = []
    mia_ship_team = {}
    for team_num in range(4):
        mia_ships += [ship_id for ship_id in cumulative_data.old_obs['players'][team_num][2] 
                      if ship_id not in cumulative_data.obs['players'][team_num][2]]
        mia_ship_team.update({ship_id:team_num for ship_id in cumulative_data.old_obs['players'][team_num][2] 
                      if ship_id not in cumulative_data.obs['players'][team_num][2]})
            
    # determine cause of death and tally up the reasons to the acting ship
    if len(mia_ships)>0:
        ms_action, ms_killer_team, ms_killer_uid  = investigate_mia(missing_ship_ids = mia_ships, 
                                                                        new_obs = cumulative_data.obs, 
                                                                        old_obs = cumulative_data.old_obs)
    
        for missing_ship in mia_ships:
            # if ship killed by ship from another team
            if (ms_killer_team[missing_ship]>-1) and (ms_killer_team[missing_ship] != mia_ship_team[missing_ship]):
                game_df.loc[game_df.ship_id == ms_killer_uid[missing_ship],'Future_ship_kills'] = (
                                game_df.loc[game_df.ship_id == ms_killer_uid[missing_ship],'Future_ship_kills'] + 1)
                    
            
            elif ms_killer_team[missing_ship] < 0: # if ship died from attacking a shipyard
                game_df.loc[game_df.ship_id == missing_ship,'Future_shipyard_kill'] = 1
            
    return game_df.append(step_df, ignore_index = True)

In [58]:
def create_ship_df_gamelevel(json_file_location):
    # Creates a pandas dataframe where each row corresponds carries key ship-level features known at a given game
    # step along with future outcome variables that we might want to predict. 
    
    with open(json_file_location) as json_file:
        gamedata = json.load(json_file)
    
    cumulative_data = CumulativeData(gamedata['steps'][0][0]['observation'])
    game_df = pd.DataFrame(create_step_features(new_obs = gamedata['steps'][0][0]['observation'], 
                                                cumulative_data = cumulative_data, 
                                                include_future = True))
    

    # for each step create step data, update future fields, and append to game_df
    for step_num in range(1,len(gamedata['steps'])):
        cumulative_data.update(new_obs = gamedata['steps'][step_num][0]['observation'])
        step_df = pd.DataFrame(create_step_features(new_obs = gamedata['steps'][step_num][0]['observation'], 
                                       cumulative_data = cumulative_data,
                                       include_future = True))
        
        game_df = game_df_update(game_df, 
                                 step_df = pd.DataFrame(create_step_features(new_obs = gamedata['steps'][step_num][0]['observation'], 
                                                                             cumulative_data = cumulative_data,
                                                                             include_future = True)), 
                                 cumulative_data = cumulative_data)
        
    return game_df

### Examples of the helpful functions

In [59]:
# Examples of cardinal_distance_list

# start at point (0,19), aka index #19, and end at point (2,3), aka index #45
start_point = 19
end_point = 45
print('The distance from cell#19 to cell#45')
print(cardinal_distance_list[start_point][end_point])
print(' ')

# start at point (2,23), aka index #45, and end at point (0,19), aka index #19
start_point = 45
end_point = 19
print('The distance from cell#45 to cell#19')
print(cardinal_distance_list[start_point][end_point])

The distance from cell#19 to cell#45
(5, 2)
 
The distance from cell#45 to cell#19
(-5, -2)


In [60]:
# Example of destination_cell

start_point = 19 # start at cell 19
move_dist = (5,2) # move 5 cells eastward and 2 cells southward

print('Destination that starts from cell 19 and moves 5 cells east and 2 cells south:')
print(destination_cell(start_point,move_dist))

Destination that starts from cell 19 and moves 5 cells east and 2 cells south:
45


In [66]:
%%time
game_df = create_ship_df_gamelevel('../kaggle_games/1950930.json')

CPU times: user 3min 25s, sys: 1.72 s, total: 3min 26s
Wall time: 4min 29s


In [65]:
game_df

,ship_id,step,team_num,board_halite_total,board_ship_total,board_shipyard_total,team_halite,team_cargo,team_ships,team_ship_births,...,action_2steps_ago,action_3steps_ago,Future_next_action,Future_life,Future_halite_delivered,Future_ship_kills,Future_shipyard_kill,Future_shipyard_defend,Future_first_convert,Future_later_convert
0,0-1,0,0,21470,4,0,5000,0,1,0,...,None,None,NORTH,1,0,0,0,0,0,1
1,0-2,0,1,21470,4,0,5000,0,1,0,...,None,None,CONVERT,0,0,0,0,0,1,0
2,0-3,0,2,21470,4,0,5000,0,1,0,...,None,None,CONVERT,0,0,0,0,0,1,0
3,0-4,0,3,21470,4,0,5000,0,1,0,...,None,None,CONVERT,0,0,0,0,0,1,0
4,0-1,1,0,20830,1,3,5000,0,1,0,...,None,None,CONVERT,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24326,59-2,399,3,23182,56,9,19617,1294,24,58,...,WEST,WEST,X,0,0,0,0,0,0,0
24327,63-2,399,3,23182,56,9,19617,1294,24,58,...,EAST,EAST,X,0,0,0,0,0,0,0
24328,8-2,399,3,23182,56,9,19617,1294,24,58,...,None,NORTH,X,0,0,0,0,0,0,0
24329,86-1,399,3,23182,56,9,19617,1294,24,58,...,EAST,None,X,0,0,0,0,0,0,0


In [67]:
game_df.to_pickle('1950930.pkl')

# Data Available

In [68]:
SCRAPED_DIR = '../kaggle_games/'
MIN_AVG_RATING = 950
episodes_df_file = 'episodes_df.pkl'

In [69]:
episode_files = []
episode_files_full = []
for dirpath, subdirs, files in os.walk(SCRAPED_DIR):
    for x in files:
        if x.endswith(".json"):
            episode_files_full.append(os.path.join(dirpath, x))
            episode_files.append(x)

episode_files = [ef.split('.')[0] for ef in episode_files]
episode_files_df = pd.DataFrame(episode_files_full,index=episode_files)
episode_files_df.drop( episode_files_df.index[[not episode_files.isdigit() for episode_files in episode_files_df.index]],inplace=True)
episode_files_df.index = [int(f) for f in episode_files_df.index]
episode_files_df = episode_files_df.loc[~episode_files_df.index.duplicated(keep='first')]
print('{} games in existing library'.format(len(episode_files_df)))

26089 games in existing library


In [70]:
all_episodes_df = pd.read_pickle(SCRAPED_DIR+episodes_df_file)
library_episodes_df = all_episodes_df.loc[all_episodes_df.id.isin(episode_files_df.index)]
select_episodes_df = library_episodes_df.drop(library_episodes_df.loc[library_episodes_df['avg_score'].values < MIN_AVG_RATING].index, inplace = False)
select_episodes_df.sort_values('avg_score', inplace = True)
select_episodes_df['file_location'] = episode_files_df.loc[select_episodes_df['id']].values
print('{} games with qualifying avg_score>'.format(len(select_episodes_df)))

23431 games with qualifying avg_score>
